In [ ]:
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service)

driver.get("https://vsmobile.bet9ja.com/mobile/themes/?sk=bet9ja&t=b61c29e6-9348-4c58-af90-378760a74693&game"
               "=league_premier&pid=14001,14003,14011,14012,14014,14015,14016,"
               "14017&v=0&text=Premier&lang=en_GB#resutls&ui_state=dialog")
# driver.maximize_window()
time.sleep(10)

action = ActionChains(driver)
record_list = []
odd_list = []

print("New Games Loading  ...\n")
while True:
    week_num = driver.find_element(By.XPATH, "//span[@id='leagueWeekNumber']").text
    counter = driver.find_element(By.XPATH, "//div[@id='bets-time-betContdown']").text

    if counter[:2] == "00" and int(counter[-2:]) < 50 and int(counter[-2:]) > 20:
    
        market_select = driver.find_element(By.XPATH, "//select[@id='bet-select-market']")

        market_select.click()
        arrow_down = [ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform() for _ in range(4)]
        action.send_keys(Keys.ENTER).perform()
        over_three = driver.find_element(By.XPATH, "//div[@id='tab_id_Over_Under_3_5']").text
        over_three = over_three.split("\n")[2::2]
        over_three = [float(x.split(" ")[3]) for x in over_three]
        # print(over_three)
        # break

        new_odd_list = []
        for i in range(1, 11):
            new_odd_list.append((i, over_three[i-1]))
        sorted_odds = sorted(new_odd_list, key=lambda x: x[1], reverse=True)
        all_odds = [x[1] for x in sorted_odds]
        for odd in sorted_odds:
            if all_odds.count(odd[1]) == 1:
                odd_list.append(odd)
                print("Odd:", odd)
                # print("WEEK:", week_num, " odd:", odd)
                break

        # print("counting")
        time.sleep(70)

        while True:
            counter = driver.find_element(By.XPATH, "//div[@id='bets-time-betContdown']").text
            current_week = driver.find_element(By.XPATH, "//span[@id='leagueWeekNumber']").text
        
            if counter[:2] == "00" and int(counter[-2:]) < 59 and int(counter[-2:]) > 50:
                week_check = int(current_week) - 1
                if week_check == 0:
                    current_week = 39
                # print("Checking Results ...")
                driver.find_element(By.XPATH, "//div[@class='ui-panel-wrapper']//i[@class='fa fa-bars icon-menu']").click()
                time.sleep(1)
                driver.find_element(By.XPATH, "//a[@id='a_bet_results']").click()
                time.sleep(8)
                try:                  
                    score_result = driver.find_element(By.XPATH, "//table[@id='results-div-header-mainTable']").text
                    driver.find_element(By.XPATH, "//div[@id='results']//i[@class='fa fa-bars icon-menu']").click()
                except ElementClickInterceptedException:
                    print("Network Delay")
                    time.sleep(20)
                    score_result = driver.find_element(By.XPATH, "//table[@id='results-div-header-mainTable']").text
                    driver.find_element(By.XPATH, "//div[@id='results']//i[@class='fa fa-bars icon-menu']").click()
                
                score_result = score_result.split(f"WEEK {int(current_week) - 1}")[1].split("\n")[1:11]
                score_result = [(int(x[4]), int(x[6])) for x in score_result][odd[0] - 1]
                print("Score -->", sum(score_result))

                time.sleep(1)
                driver.find_element(By.XPATH, "//li[@class='li_bet']//a[@id='a_bet_bet']").click()
                time.sleep(3)
                break
            else:
                time.sleep(1)   


        # data = (int(week_num), odd[0], odd[1], score_result[0], score_result[1])
        # # print(data)
        # record_list.append(data)
            
        # print("Week", int(current_week) - 1, "Data Collection Complete ...")

        # if int(current_week) == 39:
        #     new_df = pd.DataFrame(record_list, columns=[
        #         "week", "odd_position_num", "odd", "H_score", "A_score"])
        #     new_df.to_csv("auto_trade.csv", index=False)
    else:
        time.sleep(1)


# # Close the browser session
# driver.quit()

In [ ]:
new_df = pd.DataFrame(record_list, columns=["week", "odd_position_num", "odd", "H_score", "A_score"])

new_df.to_csv("auto_trade_records/rec_4.csv", index=False)